# J = 1 Hamiltonian - Find eigenvalues with VQE algorithm from PartC

In [15]:
import numpy as np

# Define the angular momentum basis states for two spin-1/2 particles
# Total angular momentum quantum number j = 1
# Allowed m values: -1, 0, 1

j = 1
m_values = [-1, 0, 1]

# Define the Hamiltonian matrix elements (for demonstration purposes)
# Replace these with the actual Hamiltonian matrix elements
H_matrix_elements = np.array([
    [2, -1, 0],
    [-1, 1, -1],
    [0, -1, 3]
])

# Initialize the linear combination string
linear_combination = "H = "

# Iterate over the matrix elements and calculate the linear combination
for i in range(len(m_values)):
    for j in range(len(m_values)):
        coefficient = H_matrix_elements[i, j]
        linear_combination += f"{coefficient} * P_{m_values[i]}_{m_values[j]} + "

# Remove the trailing '+' and whitespace
linear_combination = linear_combination[:-3]

# Print the linear combination
print(linear_combination)

import numpy as np

# Define the individual spin-1/2 basis states
up = np.array([1, 0])
down = np.array([0, 1])

# Define the basis states for total angular momentum j=1
# Calculate the tensor product to get the basis states
j = 1
m_values = [-1, 0, 1]
basis_states = []

for m in m_values:
    if m == -1:
        basis_states.append(np.kron(down, up))
    elif m == 0:
        basis_states.append(np.sqrt(1/2) * (np.kron(up, down) + np.kron(down, up)))
    elif m == 1:
        basis_states.append(np.kron(up, down))

# Print the vector representation of the basis states
for m, state in zip(m_values, basis_states):
    print(f'|{j},{m}⟩ = {state}')

H = 2 * P_-1_-1 + -1 * P_-1_0 + 0 * P_-1_1 + -1 * P_0_-1 + 1 * P_0_0 + -1 * P_0_1 + 0 * P_1_-1 + -1 * P_1_0 + 3 * P_1_1
|1,-1⟩ = [0 0 1 0]
|1,0⟩ = [0.         0.70710678 0.70710678 0.        ]
|1,1⟩ = [0 1 0 0]


In [16]:
import numpy as np
from itertools import product

# Define Pauli matrices
I = np.array([[1, 0], [0, 1]])
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])

# add to lisst
paulis = [I, X, Y, Z]
pauli_labels = ['I', 'X', 'Y', 'Z']

def decompose_matrix(H):
    dim = H.shape[0]
    
    # special case for dim = 2
    if dim == 2:
        tensor_products = paulis
        tensor_product_labels = pauli_labels
    else:
        # create a list of tensor products. Products are sorted for 4 dimensions by II; IX; IY; IZ; XI; XX; XY; XZ...
        tensor_products = [np.kron(p1, p2) for p1, p2 in product(paulis, repeat=int(np.log2(dim)))]
        tensor_product_labels = ['{} ⊗ {}'.format(label1, label2) for label1, label2 in product(pauli_labels, repeat=int(np.log2(dim)))]
    
    # perform the decomposition according to formula
    coefficients = []
    for product_matrix in tensor_products:
        coefficient = np.trace(np.dot(H, product_matrix)) / dim  # Normalize by the dimension of the matrix
        coefficients.append(coefficient)
    
    # Construct the linear combination string
    linear_combination = "H = "
    for coeff, label in zip(coefficients, tensor_product_labels):
        if coeff != 0:
            linear_combination += f"{coeff:.2f} * {label} + "
    
    # Remove the trailing '+' and whitespace
    linear_combination = linear_combination[:-3]
    
    return linear_combination

# define hamiltonians
e = 1; V = 1
H_2x2 = np.array([[-e, -V], [-V, e]])
Hx = 2; Hz = 3; energiesNoninteracting = [0, 2.5, 6.5,7]
H = np.diag(energiesNoninteracting) + Hx * np.kron(X,X) + Hz * np.kron(Z,Z)

print("Decomposition for 2x2 matrix:")
print(decompose_matrix(H_2x2))

print("\nDecomposition for 4x4 matrix:")
print(decompose_matrix(H))

Decomposition for 2x2 matrix:
H = -1.00 * X + -1.00 * Z

Decomposition for 4x4 matrix:
H = 4.00 * I ⊗ I + -0.75 * I ⊗ Z + 2.00 * X ⊗ X + -2.75 * Z ⊗ I + 2.50 * Z ⊗ Z


In [1]:
import qiskit as qk
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
import matplotlib.pyplot as plt
import numpy as np
from qiskit import transpile
from scipy.optimize import minimize_scalar, minimize

simulator = AerSimulator()

# define a function to create a ansatz wave function
# takes in a quantum circuit, applies the rotation around x by theta and then around y by phi
def ansatz(qc, qr, theta, phi):
    qc.rx(theta, qr[0])
    qc.ry(phi, qr[0])
    return qc
# define a function that makes measurements depending on what Operator should be measured
def measurements(qc, qr, cr, op):
    # measurement of Z operator. Nothing has to be changed, as our measurement basis are eigenstates of the operator
    if op == "Z":        
        # Measurement of qubit 0 on classical register 0
        qc.measure(qr[0],cr[0])
    # Measurement of X operator. Our measurement basis is not eigenstate of X, therfore we have to transform our 
    # measurement states with the Hadamard Gate.
    elif op == "X":
        # Change of basis, since X = HZH
        qc.h(qr[0])
        # Measurement of qubit 0 on classical register 0
        qc.measure(qr[0],cr[0])   
    return qc
# function for getting the energy by evaluating the expectation values of the hamiltonian. The expectation values
# are stored in a dictionary params#
def hamiltonian(params):
    e = 1; V = 1
    # H = (epsilon)*I-V*X
    en = e*1 -V*params["X"]
    return en


def vqe_step(theta, phi, verbose = True):
    # Number of executions for each quantum circuit
    shots=10000
    
    vqe_res = dict()
    qc_list = dict()
    
    for op in ["X", "Z"]:
        qr = qk.QuantumRegister(1, "qr")
        cr = qk.ClassicalRegister(1, "cr")
        qc = qk.QuantumCircuit(qr, cr)

        # Implementation of the ansatz
        qc = ansatz(qc, qr, theta, phi)

        # Just for plotting purposes
        qc.barrier()

        # Measurements in the appropriate basis (X,Z) are implemented
        qc = measurements(qc, qr, cr, op)
                
        # Get the measurements results
        #counts = qk.execute(qc, simulator, shots=shots).result().get_counts()
        new_circuit = transpile(qc, simulator)
        job = simulator.run(new_circuit, shots = shots)
        counts = job.result().get_counts(qc)
        # Check the results, and evaluate the mean value dividing by the number of shots
        if len(counts) == 1: 
            try:
                counts['0']
                mean_val = 1
            except:
                mean_val = -1
        else:
            # Evaluates the mean value of Z operator, as the difference in the number of 
            # 0s and 1s in the measurement outcomes
            mean_val = (counts['0']-counts['1'])/shots
            
        vqe_res[op] = mean_val
        qc_list[op] = qc
        
    energy = hamiltonian(vqe_res)
    
    if verbose: 
        print("Mean values from measurement results:\n", vqe_res) 
        print(f"\n{'Theta':<10} {'Energy':<10} {'<X>':<10} {'<Z>':<10}")
        print(f"{theta:<10f} {energy:<10f} {vqe_res['X']:<10f} {vqe_res['Z']:<10f}")
    
        return energy, qc_list
    
    else: 
        return energy

In [2]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate Eigenvalues with Pauli Matrices
X = np.array([[0, 1], [1, 0]])
Y = np.array([[0, -1j], [1j, 0]])
Z = np.array([[1, 0], [0, -1]])
I = np.array([[1, 0], [0, 1]])
e = 1; V = 1

H = e * I -V * X
print("Eigenvalue with Numpy: ", np.linalg.eig(H)[0])

# Gradient descent parameters
eta = 0.05
Niterations = 60
energies = []
shots = 10000
# Random angles using uniform distribution
theta = 2 * np.pi * np.random.rand()
phi = 2 * np.pi * np.random.rand()
pi2 = 0.3 * np.pi
# Perform gradient descent
for iter in range(Niterations):
    
    thetagradient = 1 / (pi2) * (vqe_step(theta + pi2, phi,shots) - vqe_step(theta - pi2, phi,shots))
    phigradient = 1 / (pi2) * (vqe_step(theta, phi + pi2,shots ) - vqe_step(theta, phi - pi2,shots))
    theta -= eta * thetagradient
    phi -= eta * phigradient
    energy = vqe_step(theta, phi,shots)
    energies.append(energy)
    print("Iteration:", iter, "Energy:", energy)

print("Minimum energy: ", vqe_step(theta, phi,shots))

# Plot the energy values
plt.scatter(range(Niterations), energies, s=1)
plt.xlabel('Iteration')
plt.ylabel('Energy')
plt.title('Energy vs Iteration')
plt.grid(True)
plt.show()

Eigenvalue with Numpy:  [2. 0.]
Mean values from measurement results:
 {'X': 0.399, 'Z': -0.5454}

Theta      Energy     <X>        <Z>       
5.455673   0.601000   0.399000   -0.545400 
Mean values from measurement results:
 {'X': -0.5384, 'Z': 0.7258}

Theta      Energy     <X>        <Z>       
3.570717   1.538400   -0.538400  0.725800  


TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'